In [1]:
!pip install datasets --quiet
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install transformers --quiet


In [10]:
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from datasets import Dataset
from inspect import signature

# --- Imports for Class Weighting ---
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch import nn
# ----------------------------------


# ========== 1. Load and preprocess data ==========
csv_path = "/content/drive/MyDrive/nda_models/Final_NDA_with_Augmented.csv"
data = pd.read_csv(csv_path, encoding="ISO-8859-1")

# Use correct columns
data = data.rename(columns={
    'clean_sentence': 'SENTENCE',
    'Classification_Category': 'CATEGORY'
})

# Clean and normalize
data = data[['SENTENCE', 'CATEGORY']].dropna()
data['CATEGORY'] = data['CATEGORY'].str.strip().str.lower()

# Map similar labels to consistent forms
category_map = {
    'confidentiality obligation': 'confidentiality obligations',
    'confidentiality obligations': 'confidentiality obligations',
    'signatures': 'signatures',
    'signature': 'signatures',
    'governing law': 'governing law',
    'remedies': 'remedies',
    'non-competition': 'non-competition',
    'non competition': 'non-competition',
    'privacy/security': 'privacy/security',
    'limitation of liability': 'limitation of liability',
    'non-solicitation': 'non-solicitation',
    'indemnification': 'indemnification'
}
data['CATEGORY'] = data['CATEGORY'].map(category_map)

# Drop invalid rows
data = data.dropna(subset=['CATEGORY']).reset_index(drop=True)

# Encode labels
label2id = {label: idx for idx, label in enumerate(sorted(data['CATEGORY'].unique()))}
id2label = {idx: label for label, idx in label2id.items()}
data['label'] = data['CATEGORY'].map(label2id)

print(f"Total samples: {len(data)} | Classes: {len(label2id)}")
print("Label mapping:", label2id)

# Split train/test
train_data, test_data = train_test_split(
    data, test_size=0.3, stratify=data['label'], random_state=42
)

# ========== 1.5. Calculate Class Weights ==========
# Use only training data to calculate weights
train_labels = train_data['label'].values
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# Convert weights to a PyTorch Tensor
weights_tensor = torch.tensor(class_weights, dtype=torch.float)
print(f"Calculated Class Weights: {weights_tensor}")
# ---------------------------------------------------------


# ========== 2. Model training loop ==========
bert_variants = [
    "bert-base-uncased",
    "roberta-base",
    "distilbert-base-uncased"
]

# ========== 2.5. Custom Weighted Trainer Definition ==========
# Define the custom trainer class once
class WeightedTrainer(Trainer):
    # --- MODIFICATION 1: Added **kwargs to accept extra arguments ---
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Pop labels from inputs
        labels = inputs.pop("labels")

        # Pass remaining inputs to the model
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Key step: Use the pre-calculated weights_tensor
        # Move weights to the same device as the model (GPU/CPU)
        loss_fct = nn.CrossEntropyLoss(weight=weights_tensor.to(model.device))

        # Calculate the weighted loss
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss
# -----------------------------------------------------------------

for model_name in bert_variants:
    print(f"\n\n===== Training with {model_name} =====")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(label2id),
        id2label=id2label,
        label2id=label2id
    )

    # Convert to HuggingFace Datasets
    train_ds = Dataset.from_pandas(train_data)
    test_ds = Dataset.from_pandas(test_data)

    def tokenize_function(example):
        return tokenizer(
            example['SENTENCE'],
            truncation=True,
            padding='max_length',
            max_length=128
        )

    train_ds = train_ds.map(tokenize_function, batched=True)
    test_ds = test_ds.map(tokenize_function, batched=True)

    train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # ========== 3. TrainingArguments (fully compatible) ==========
    _args = {
        "output_dir": f"./results_{model_name.replace('/', '_')}",
        "per_device_train_batch_size": 8,
        "per_device_eval_batch_size": 8,
        "num_train_epochs": 2,
        "weight_decay": 0.01,  # This is L2 regularization, not class weight
        "logging_steps": 50,
        "evaluation_strategy": "epoch",
        "eval_strategy": "epoch",
        "save_strategy": "epoch",
        "load_best_model_at_end": False,
        "metric_for_best_model": "accuracy",
        "report_to": "none",
        "do_eval": True
    }

    # Keep only keys supported by the TrainingArguments constructor
    valid_keys = set(signature(TrainingArguments.__init__).parameters.keys())
    _args = {k: v for k, v in _args.items() if k in valid_keys}

    training_args = TrainingArguments(**_args)

    # Optional: add eval_steps for very old versions
    extra_trainer_kwargs = {}
    if "eval_steps" in signature(Trainer.__init__).parameters:
        extra_trainer_kwargs["eval_steps"] = 500

    # ========== 4. Compute metrics ==========
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, predictions)
        return {"accuracy": acc}

    # ========== 5. Trainer (using WeightedTrainer) ==========
    trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        # --- MODIFICATION 2: Removed deprecated `tokenizer=tokenizer` ---
        **extra_trainer_kwargs
    )

    trainer.train()

    # ========== 6. Evaluation ==========
    preds = trainer.predict(test_ds).predictions
    pred_labels = np.argmax(preds, axis=1)
    true_labels = test_data['label'].tolist()
    acc = accuracy_score(true_labels, pred_labels)

    print(f"\n>>> Test Accuracy ({model_name}): {acc * 100:.2f}%")
    print(classification_report(true_labels, pred_labels,
                                target_names=[id2label[i] for i in sorted(id2label)]))

    # ========== 7. Save model & tokenizer ==========
    # The 'weighted' name is now accurate
    save_dir = f"/content/drive/MyDrive/nda_models/fine_tuned_{model_name.replace('/', '_')}_weighted"

    trainer.save_model(save_dir)
    tokenizer.save_pretrained(save_dir)
    print(f"Saved fine-tuned model and tokenizer to {save_dir}")

    # ========== 8. Save predictions ==========
    # (Uncomment if needed)
    # pred_df = pd.DataFrame({
    #     "sentence": test_data["SENTENCE"].tolist(),
    #     "true_label": [id2label[i] for i in true_labels],
    #     "pred_label": [id2label[i] for i in pred_labels]
    # })
    # csv_path = f"./predictions_{model_name.replace('/', '_')}_weighted.csv"
    # pred_df.to_csv(csv_path, index=False, encoding="utf-8")
    # print(f"Saved predictions to {csv_path}")



Total samples: 6258 | Classes: 7
Label mapping: {'confidentiality obligations': 0, 'governing law': 1, 'indemnification': 2, 'non-competition': 3, 'non-solicitation': 4, 'remedies': 5, 'signatures': 6}
Calculated Class Weights: tensor([ 0.1756,  3.4956, 10.0922,  7.5387, 10.4286,  2.2032,  4.1994])


===== Training with bert-base-uncased =====


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/1878 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.171400,0.239429,0.974973
2,0.121400,0.240088,0.976038


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4380 [00:00<?, ? examples/s]

Map:   0%|          | 0/1878 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.145000,0.236679,0.966986
2,0.063200,0.185655,0.975506



>>> Test Accuracy (distilbert-base-uncased): 97.55%
                             precision    recall  f1-score   support

confidentiality obligations       0.99      0.98      0.99      1527
              governing law       0.96      1.00      0.98        77
            indemnification       1.00      1.00      1.00        27
            non-competition       0.86      0.89      0.88        36
           non-solicitation       0.96      0.88      0.92        26
                   remedies       0.92      0.93      0.93       121
                 signatures       0.85      0.95      0.90        64

                   accuracy                           0.98      1878
                  macro avg       0.93      0.95      0.94      1878
               weighted avg       0.98      0.98      0.98      1878

Saved fine-tuned model and tokenizer to /content/drive/MyDrive/nda_models/fine_tuned_distilbert-base-uncased_weighted
